In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs
from tf_lab.point_clouds.ae_templates import conv_architecture_ala_nips_17, default_train_params_ala_nips_17
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry, decoder_with_fc_only
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from pc_completions.evaluation import basic_comletion_measures

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_samples = 2048
do_training = True
first_time_running = False
load_model = True
seed = 42
max_training_epochs = 2000
loss = 'chamfer'
z_rotate = True
experiment_name = '1_all_snc_rotated_mlp_arch_with_bnorm_' + str(n_pc_samples) +  'pts_' + loss

In [5]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/all_snc', experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/1_all_snc_rotated_mlp_arch_with_bnorm_2048pts_chamfer'

In [6]:
n_threads = 40
verbose = True
full_file_names = [f for f in files_in_subdirs(full_pclouds_path, '.ply')]
pclouds, model_names, syn_ids = load_point_clouds_from_filenames(full_file_names, n_threads, snc_loader, verbose)
model_unames = model_names + '.' + syn_ids
train_data = PointCloudDataSet(pclouds, labels=model_unames)

/orions4-zfs/projects/optas/Git_Repos/tf_lab/point_clouds/in_out.py:38: UserWarning: Point clouds with the same model name were loaded.
  warnings.warn('Point clouds with the same model name were loaded.')


57449 pclouds were loaded. They belong in 57 shape-classes.


In [7]:
n_input = [n_pc_samples, 3]

encoder_args = {'n_filters': [64, 128, 128, 256, 512],
                'filter_sizes': [1, 1, 1, 1, 1],
                'strides': [1, 1, 1, 1, 1],
                'b_norm': True,
                'dropout_prob': [0, 0, 0, 0, 0.5]
               }

decoder_args = {'layer_sizes': [512, 1024, np.prod(n_input)],
                'b_norm': True
               }

if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    
else:    
    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                z_rotate = z_rotate,
                encoder = encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = decoder_with_fc_only,
                decoder_args = decoder_args,            
               )
    conf.allow_gpu_growth = False
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))

              allow_gpu_growth: False
                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'b_norm': True, 'layer_sizes': [512, 1024, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'filter_sizes': [1, 1, 1, 1, 1], 'strides': [1, 1, 1, 1, 1], 'n_filters': [64, 128, 128, 256, 512], 'dropout_prob': [0, 0, 0, 0, 0.5], 'b_norm': True}
               experiment_name: 1_all_snc_rotated_mlp_arch_with_bnorm_2048pts_chamfer
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None


In [ ]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = saved_epochs[-1]
    ae.restore_model(conf.train_dir, last_epoch, verbose=True)

Model restored in epoch 1620.


In [ ]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '1621', 'training time (minutes)=', '2.3675', 'loss=', '0.000397231')
('Epoch:', '1622', 'training time (minutes)=', '2.3976', 'loss=', '0.000392704')
('Epoch:', '1623', 'training time (minutes)=', '2.4044', 'loss=', '0.000407782')
('Epoch:', '1624', 'training time (minutes)=', '2.4173', 'loss=', '0.000397902')
('Epoch:', '1625', 'training time (minutes)=', '2.4106', 'loss=', '0.000391892')
('Epoch:', '1626', 'training time (minutes)=', '2.4098', 'loss=', '0.000394281')
('Epoch:', '1627', 'training time (minutes)=', '2.4165', 'loss=', '0.000392725')
('Epoch:', '1628', 'training time (minutes)=', '2.4112', 'loss=', '0.000391491')
('Epoch:', '1629', 'training time (minutes)=', '2.4078', 'loss=', '0.000394983')
('Epoch:', '1630', 'training time (minutes)=', '2.4172', 'loss=', '0.000404344')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/1_all_snc_rotated_mlp_arch_with_bnorm_2048pts_chamfer/models.ckpt-1630 is not in all_model_checkpoint_paths. Manua